# Waze Partner Feed Data API

In [18]:
import requests, json, os
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns; sns.set()
from matplotlib.path import Path as mpl_path
from datetime import datetime

### Waze partner feed methods

In [36]:
waze_url = 'https://www.waze.com/partnerhub-api/waze-feed-access-token/c37c11ba-ff9d-4ad5-8ecc-4e4f12e91efb?format=1'

# Load custers polygons geojson data
polygons_geojson_path = '../Hackaton COR IV - Centro de Operações do RJ/ACELERAÇÃO/Dados/Clusters/polygons_micro.geojson'
polygons_geojson = json.loads(open(polygons_geojson_path, 'r').read())

# Get polygons dict from geojson
polygons = {}
for poly in polygons_geojson['features']:
    if poly['properties']['sublabel'] != -1:
        polygons[poly['properties']['sublabel']] = poly['geometry']['coordinates'][0]

def get_waze_partner_alerts(alert_type='alerts'):
    "alert_type: one of 'alerts', 'irregularities', 'jams'"
    incidents = requests.get(waze_url).json()
    if alert_type not in incidents.keys():
        return None
    df = pd.DataFrame(incidents[alert_type])
    # Data cleaning & preprocessing
    if 'location' in df.columns:
        df[['latitude', 'longitude']] = list(df['location'].map(lambda coords: [coords['x'], coords['y']]))
        df.drop('location', axis=1, inplace=True)
    if 'pubMillis' in df.columns:
        df['pubMillis'] = (df['pubMillis'] / 1000).map(datetime.fromtimestamp)
        df.sort_values('pubMillis', ascending=False, inplace=True)
        df.reset_index(drop=True, inplace=True)
    return df

def get_waze_partner_alerts_extended():
    # Get waze alerts
    alerts = get_waze_partner_alerts(alert_type='alerts')
    # Get events points array
    points = np.array(list(map(tuple, alerts[['latitude', 'longitude']].values)))
    # Get clusters polygons events dict
    events_poly = {}
    for cluster_id, poly in polygons.items():
        mpl_poly =  mpl_path(poly)
        points_msk = mpl_poly.contains_points(points)
        poly_events_df = alerts[points_msk]
        poly_events_ids = list(poly_events_df['uuid'])
        for event_id in poly_events_ids:
            events_poly[event_id] = cluster_id
    
    # Update events dataframe with events polygons ids
    alerts['cluster_id'] = -1
    for event_id, cluster_id in events_poly.items():
        alerts['cluster_id'][alerts['uuid']==event_id] = cluster_id

    # Return extended open events
    return alerts

def get_clusters_waze_partner_alerts():
    # Get waze alerts
    alerts = get_waze_partner_alerts(alert_type='alerts')
    # Get events points array
    points = np.array(list(map(tuple, alerts[['latitude', 'longitude']].values)))
    # Get clusters polygons events dict
    poly_events = []
    for cluster_id, poly in polygons.items():
        mpl_poly =  mpl_path(poly)
        points_msk = mpl_poly.contains_points(points)
        poly_events_df = alerts[points_msk]
        poly_events_ids = list(poly_events_df['uuid'])
        poly_wb_events_ids = list(poly_events_df['uuid'][poly_events_df['subtype']=="HAZARD_WEATHER_FLOOD"])
        poly_events.append({
            'cluster_id': cluster_id,
            'alerts': len(poly_events_ids),
            'alerts_ids': poly_events_ids,
            'alerts_status': int(bool(len(poly_events_ids))),
            'waterbag_alerts': len(poly_wb_events_ids),
            'waterbag_alerts_ids': poly_wb_events_ids,
            'waterbag_alerts_status': int(bool(len(poly_wb_events_ids))),
        })
    return pd.DataFrame(poly_events)

---
## Waze partner feed request

In [12]:
alerts = get_waze_partner_alerts(alert_type='alerts')

alerts.head()

,country,city,reportRating,confidence,reliability,type,uuid,roadType,magvar,subtype,street,pubMillis,nThumbsUp,reportDescription,latitude,longitude
0,BR,NaN,0,0,5,ACCIDENT,302e57f6-73c6-4547-9fe5-41eaf69af136,6.0,73,ACCIDENT_MINOR,BR-116 S Rod. Pres. Dutra,2022-11-13 01:22:30,NaN,NaN,-43.822723,-22.656735
1,BR,Rio de Janeiro,2,0,6,JAM,7ef7f1ca-48e4-4ef4-be5d-347e91f1c878,2.0,0,JAM_STAND_STILL_TRAFFIC,Av. Ayrton Senna - Pista Central,2022-11-13 01:21:55,0.0,NaN,-43.365471,-22.980718
2,BR,Rio de Janeiro,3,0,5,WEATHERHAZARD,10a1abfa-955e-4200-812a-19268e1c864b,6.0,242,HAZARD_ON_SHOULDER_CAR_STOPPED,Av. Brasil,2022-11-13 01:21:55,NaN,NaN,-43.611198,-22.874937
3,BR,Rio de Janeiro,3,0,5,JAM,c4919d4e-af68-44cd-94ae-e716e3313269,2.0,348,JAM_HEAVY_TRAFFIC,NaN,2022-11-13 01:21:33,NaN,NaN,-43.365433,-22.979868
4,BR,Rio de Janeiro,3,0,6,JAM,0c0604cc-439c-4031-862b-703cb579effc,2.0,160,JAM_MODERATE_TRAFFIC,Vd. São Sebastião,2022-11-13 01:21:26,0.0,NaN,-43.197115,-22.908353


## Clusters polygons waze alerts

### Waze partner alerts extended

In [35]:
clusters_waze_alerts = get_waze_partner_alerts_extended()

clusters_waze_alerts.head()

<ipython-input-33-441539a9748f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alerts['cluster_id'][alerts['uuid']==event_id] = cluster_id


,country,city,reportRating,confidence,reliability,type,uuid,roadType,magvar,subtype,street,pubMillis,nThumbsUp,reportDescription,latitude,longitude,cluster_id
0,BR,Rio de Janeiro,3,0,5,WEATHERHAZARD,11c754c1-39e5-4f7c-b9c8-aac1f0ee359a,2.0,280,HAZARD_ON_ROAD_CAR_STOPPED,NaN,2022-11-13 01:42:27,NaN,NaN,-43.220739,-22.888744,-1
1,BR,Imbariê (Duque de Caxias),0,0,5,JAM,67422705-64cf-4636-8e93-80919b25a5b5,1.0,359,JAM_STAND_STILL_TRAFFIC,R. A,2022-11-13 01:40:48,NaN,NaN,-43.251142,-22.650578,-1
2,BR,Rio de Janeiro,2,1,7,ACCIDENT,9a9491c7-88bf-4f09-8f6a-9b5e1486cdbb,6.0,272,ACCIDENT_MINOR,Av. Brasil - Pista Central,2022-11-13 01:38:41,NaN,NaN,-43.221011,-22.888761,-1
3,BR,Rio de Janeiro,2,0,5,JAM,30d9305d-5fe9-4113-8936-b9e3e90a1732,2.0,70,,Av. Pres. Vargas,2022-11-13 01:35:30,NaN,NaN,-43.200138,-22.908385,-1
4,BR,Rio de Janeiro,2,0,5,JAM,c9e0cbe0-896c-4c26-aa7a-f53db0ee1303,2.0,150,JAM_HEAVY_TRAFFIC,R. Carmo Neto,2022-11-13 01:34:59,NaN,NaN,-43.200502,-22.908733,-1


### Clusters polygons waze alerts

In [37]:
clusters_waze_alerts = get_clusters_waze_partner_alerts()

clusters_waze_alerts.head()

,cluster_id,alerts,alerts_ids,alerts_status,waterbag_alerts,waterbag_alerts_ids,waterbag_alerts_status
0,-1,219,"[912ef77a-ccf3-453a-9f3a-da21ce5348bc, 5482df7...",1,2,"[32229b73-54c9-406d-95b9-aeb7e392a0fb, 894904c...",1
1,0,0,[],0,0,[],0
2,1,0,[],0,0,[],0
3,2,0,[],0,0,[],0
4,3,0,[],0,0,[],0


In [42]:
clusters_waze_alerts.set_index('cluster_id').drop(-1)['alerts'].sort_values(ascending=False)

cluster_id
15    2
61    1
54    1
60    0
58    0
     ..
26    0
25    0
24    0
23    0
82    0
Name: alerts, Length: 79, dtype: int64